In [13]:
# download protein sequences and CDD domain annotation from ensembl biomart

protein_file="GRCh38.p13-ensembl-biomart-protein-20220922.txt"
'''
>ENSP00000005374
MANSGCKDVTGPDEESFLYFAYGSNLLTERIHLRNPSAAFFCVARLQDFKLDFGNSQGKT
SQTWHGGIATIFQSPGDEVWGVVWKMNKSNLNSLDELFAWVQKKMVCRWSIKRS*
'''

# dfile=GRCh38.p13-ensembl-biomart-CDD-coordinates-20220922.txt
#  sort -k4 -k2n $dfile > $dfile.sorted 
domain_file="GRCh38.p13-ensembl-biomart-CDD-coordinates-20220922.txt.sorted"
# sort by coordinates
#
'''
ENSG00000004059 18      176     ENSP00000000233 ARF5
ENSG00000173153 73      168     ENSP00000000442 ESRRA
ENSG00000173153 197     420     ENSP00000000442 ESRRA
ENSG00000001617 48      548     ENSP00000002829 SEMA3F
ENSG00000001617 609     697     ENSP00000002829 SEMA3F
'''


'\nENSG00000004059 18      176     ENSP00000000233 ARF5\nENSG00000173153 73      168     ENSP00000000442 ESRRA\nENSG00000173153 197     420     ENSP00000000442 ESRRA\nENSG00000001617 48      548     ENSP00000002829 SEMA3F\nENSG00000001617 609     697     ENSP00000002829 SEMA3F\n'

In [29]:
# load sequences
from Bio import SeqIO

proteins = {}
for record in SeqIO.parse(protein_file, "fasta"):
    proteins[str(record.id)] = str(record.seq)

# save as single line text
outf = open('protein.txt','w')
for p in proteins:
    outf.write(proteins[p][:-1]+'\n')
outf.close()

In [41]:
# get interdomain sequences
# also get C-termini that are within domains 
unfolded_cterm={} # the last domain sequences, if less than 10 aa from the end of protein
previous_protein=''
previous_start=0
previous_end = 0
linkers={}
with open(domain_file,'r') as f:
    n=0
    for line in f:
        n=n+1
        print(n,'\r',end='')
        #print(line)
        flds = line.strip().split()
        protein = flds[3]
        start=int(flds[1])
        end=int(flds[2])
        if protein == previous_protein: # the same protein
            # calculate interdomain coordinates
            if start > previous_end:
                linkers[proteins[protein][previous_end:start]] = protein+'\t'+str(previous_end)+'-'+str(start)
        else: # a new protein
            # is the previous protein folded at C-termini?
            if previous_protein != '':
                d_to_end = len(proteins[previous_protein])-previous_end
                if d_to_end > 100:
                    unfolded_cterm[proteins[previous_protein][previous_end:-1]] = protein+'\t'+str(d_to_end)
        previous_protein = protein
        previous_end = end
        previous_start = start
                    
f.close()

In [42]:
# save
outf = open('linkers.txt','w')
for seq in linkers:
    outf.write('\t'.join([seq,str(len(seq)),linkers[seq]])+'\n')
outf.close()
# save
outf = open('unfolded_cterm.txt','w')
for seq in unfolded_cterm:
    outf.write('\t'.join([seq,str(len(seq)),unfolded_cterm[seq]])+'\n')
outf.close()

{'PLPFPGPFPAGPLAVAGGPRKTAAPVNAL': 'ENSP00000384851:168-197',
 'QAYGAACADCCLARDPYCAWDGQACSRYTASSKRRSRRQDVRHGNPIRQCRGFNSNANKNA': 'ENSP00000409859:517-578',
 'DFLQKQEYKTLEYNLTT': 'ENSP00000514471:372-389',
 'LEGDAPVSWTETKKQSFKQTGEFGEKRKNSILNPINSIRKFSIVQKTPLQMNGIEEDSDEPLERRLSLVPDSEQGEAILPRISVISTGPTLQARRRQSVLNLMTHSVNQGQNIHRKTTASTRKVSLAPQANLTELDIYSRRLSQETGLEISEEINEEDLKECFFDDMESIPAVT': 'ENSP00000514471:670-844',
 'DMPTEGKPTKSTKPYKNGQLSKVMIIENSHVKKDDIWPSGG': 'ENSP00000514473:1025-1066',
 'EPSQDTVATEPSEVEGSAANKEVLAKVIDLTHDNKDDLQAAIALSLLESPKIQADGRDLNRMHEATSAETKRSKRKRCEVWGENPNPNDWRRVDGWPVG': 'ENSP00000513009:64-163',
 'NDKLPYFNAEAAPTESDQMSEVEALSVELKHYIQEDNWRFEQEVEEWEEEQSCKIPQMESSTNSSSQDYSTSQEPSVASSHGVRCLSSEHAVIVKEQTAQAIANTARAYEKSGVEAALSEVMLSPAMQGVILAIAKARQTFDRDGSEAGL': 'ENSP00000003302:648-798',
 'RSRKL': 'ENSP00000437128:91-96',
 'HIGLIPVKSSPDEPLTWQYVDQFVSESGGVRGSLGSPGNRENKE': 'ENSP00000437128:176-220',
 'AAAGRELFMTDRERLAEARQRELQRQELLMQKRLAMESNKILQEQQEMERQRRKEIAQKAAEENERYRKEMEQIVEEEEKFKKQWEEDWGS